In [1]:
#Library Declaration
import numpy as np
import scipy as sp
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import os
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn import preprocessing

In [2]:
#convert clean data into dataframe
df = pd.read_csv('C:/Users/Prateek/Downloads/grp13p2/cleanedgrp13.csv')
df = df.drop(['Unnamed: 0'],axis=1) #dropping the first column which is not required 
                                    #and gets added while saving as csv when preprocessing
df.head()


,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,122,0,1,0,1,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,122,0,1,0,1,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,122,0,1,0,1,2,2,0,94.5,...,152,5,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164,1,1,0,0,3,1,0,99.8,...,109,5,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164,1,1,0,0,3,0,0,99.4,...,136,5,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


In [3]:
#Extracting our target variable or variable that we have to predict
target = df['symboling']+2 #adding 2 to make the range 0 to 5
print("Target Shape",target.shape) #Check the shape of target variable

#Extract features from our dataframe
features = df[df.columns[1:26]] #extracting all the columns except symboling
print("Features shape:",features.shape) #Check feature shape

features=features.to_numpy(dtype ='float64')

Target Shape (204,)
Features shape: (204, 25)


In [4]:
#Normalizing our data using standardscaler
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [5]:
#Use one hot encoding to convert target variable into encoded vector
yenc = tf.keras.utils.to_categorical(target , num_classes=6)
print("shape of label array",yenc.shape)
#we represent symboling value by an array as shown below with the value for a particular point 
#indicated by the index where 1 is present.(value being 0 to 5)
#which we will convert to (-2 to 3) later.
#Thee arrays will be used as labels
#There are 204 sample points in given data and corresponding 6 lenght array for its symboling value

shape of label array (204, 6)


In [21]:
#Split data into train and test 
X_train, X_test, y_train, y_test = train_test_split(features, yenc, test_size=0.20, random_state=42)

In [22]:
#Neural network model
model = Sequential()
model.add(Dense(20, input_shape=(X_train.shape[1],), activation="relu"))
model.add(Dense(6, activation="sigmoid"))
#Description of model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 20)                520       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 126       
Total params: 646
Trainable params: 646
Non-trainable params: 0
_________________________________________________________________


In [23]:
#using categorical_cross_entropy as loss function
model.compile(loss="categorical_crossentropy", optimizer="Rmsprop", metrics=["accuracy"])

In [24]:
#creating logs for storing each epoch metrics
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(logdir)

#using earlystopping for stopping at convergence of accuracy/loss
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size = 51,epochs = 500,verbose=1,callbacks=[tensorboard, early_stopping])

Epoch 1/500
4/4 [==============================] - 0s 52ms/step - loss: 1.8514 - accuracy: 0.3006 - val_loss: 2.2059 - val_accuracy: 0.2683
Epoch 2/500
4/4 [==============================] - 0s 7ms/step - loss: 1.7448 - accuracy: 0.3620 - val_loss: 2.1421 - val_accuracy: 0.2927
Epoch 3/500
4/4 [==============================] - 0s 6ms/step - loss: 1.6846 - accuracy: 0.3926 - val_loss: 2.0814 - val_accuracy: 0.2927
Epoch 4/500
4/4 [==============================] - 0s 7ms/step - loss: 1.6351 - accuracy: 0.4294 - val_loss: 2.0489 - val_accuracy: 0.2927
Epoch 5/500
4/4 [==============================] - 0s 6ms/step - loss: 1.5952 - accuracy: 0.4356 - val_loss: 2.0057 - val_accuracy: 0.2927
Epoch 6/500
4/4 [==============================] - 0s 8ms/step - loss: 1.5598 - accuracy: 0.4540 - val_loss: 1.9565 - val_accuracy: 0.3171
Epoch 7/500
4/4 [==============================] - 0s 6ms/step - loss: 1.5210 - accuracy: 0.4540 - val_loss: 1.9344 - val_accuracy: 0.3171
Epoch 8/500
4/4 [=========

4/4 [==============================] - 0s 6ms/step - loss: 0.8418 - accuracy: 0.7546 - val_loss: 1.2002 - val_accuracy: 0.5854
Epoch 60/500
4/4 [==============================] - 0s 6ms/step - loss: 0.8355 - accuracy: 0.7546 - val_loss: 1.1970 - val_accuracy: 0.5854
Epoch 61/500
4/4 [==============================] - 0s 6ms/step - loss: 0.8286 - accuracy: 0.7607 - val_loss: 1.1889 - val_accuracy: 0.5854
Epoch 62/500
4/4 [==============================] - 0s 6ms/step - loss: 0.8228 - accuracy: 0.7669 - val_loss: 1.1979 - val_accuracy: 0.5854
Epoch 63/500
4/4 [==============================] - 0s 6ms/step - loss: 0.8179 - accuracy: 0.7669 - val_loss: 1.1856 - val_accuracy: 0.5854
Epoch 64/500
4/4 [==============================] - 0s 6ms/step - loss: 0.8116 - accuracy: 0.7607 - val_loss: 1.1820 - val_accuracy: 0.5854
Epoch 65/500
4/4 [==============================] - 0s 5ms/step - loss: 0.8042 - accuracy: 0.7669 - val_loss: 1.1603 - val_accuracy: 0.5854
Epoch 66/500
4/4 [===============

In [43]:
%load_ext tensorboard
%tensorboard --logdir="logs"
#graphs showing epoch accuracy and loss

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [26]:
df1 = pd.read_csv('C:/Users/Prateek/Downloads/cleanedwoen1.csv')
df1 = df1.drop(['Unnamed: 0'],axis=1)
df1

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,122,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,122,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,122,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,16845.0
200,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19,25,19045.0
201,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134.0,5500.0,18,23,21485.0
202,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106.0,4800.0,26,27,22470.0


In [27]:
index=[] #using the file without encoded values to encode the non numerical values in test data
trans=[]
head = ["make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","engine-type",
         "num-of-cylinders","fuel-system"]
for i in head:
    index.append(df1.columns.get_loc(i))
    le = preprocessing.LabelEncoder()
    le.fit(df1[i])
    trans.append(le)

In [28]:
#test point to be classified
test= np.array([93,'mercedes-benz','diesel','turbo','two','hardtop','rwd','front',106.70,187.50,70.30,54.90,3495,'ohc','five',183,'idi',3.58,3.64,21.50,123,4350,22,25,28176])

In [29]:
for i in range(len(trans)):
    a=index[i]
    a=a-1
    test[a]=trans[i].transform([test[a]])[0] #encoding non numerical vallues to integers

In [30]:
test

array(['93', '9', '0', '1', '1', '1', '2', '0', '106.7', '187.5', '70.3',
       '54.9', '3495', '3', '1', '183', '3', '3.58', '3.64', '21.5',
       '123', '4350', '22', '25', '28176'], dtype='<U13')

In [31]:
testnew = test
testnew = testnew[np.newaxis,:]
testnew1 = scaler.transform(testnew)
mp=model.predict(testnew1)
print((mp))

[[0. 0. 1. 0. 0. 0.]]


In [32]:
testnew.shape

(1, 25)

In [33]:
print('symboling value of the test point is',np.argmax(mp)-2)

symboling value of the test point is 0


# Model with regularizer

In [65]:
model1 = Sequential()
model1.add(Dense(20, input_shape=(X_train.shape[1],), activation="relu",kernel_regularizer=tf.keras.regularizers.l1(0.01)))
model1.add(Dense(6, activation="sigmoid"))  #adding l1 regularizer
#Description of our model
model1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 20)                520       
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 126       
Total params: 646
Trainable params: 646
Non-trainable params: 0
_________________________________________________________________


In [66]:
model1.compile(loss="categorical_crossentropy", optimizer="RMSProp", metrics=["accuracy"])

In [67]:
logdir1 = os.path.join("logs1", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard1 = TensorBoard(logdir1)
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)
model1.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size = 51,epochs = 500,verbose=1,callbacks=[tensorboard1, early_stopping])

Epoch 1/500
4/4 [==============================] - 0s 44ms/step - loss: 2.6350 - accuracy: 0.3067 - val_loss: 2.6767 - val_accuracy: 0.2683
Epoch 2/500
4/4 [==============================] - 0s 7ms/step - loss: 2.5320 - accuracy: 0.3313 - val_loss: 2.6327 - val_accuracy: 0.2683
Epoch 3/500
4/4 [==============================] - 0s 7ms/step - loss: 2.4724 - accuracy: 0.3742 - val_loss: 2.6007 - val_accuracy: 0.2927
Epoch 4/500
4/4 [==============================] - 0s 6ms/step - loss: 2.4190 - accuracy: 0.3865 - val_loss: 2.5607 - val_accuracy: 0.3171
Epoch 5/500
4/4 [==============================] - 0s 7ms/step - loss: 2.3752 - accuracy: 0.4049 - val_loss: 2.5370 - val_accuracy: 0.3415
Epoch 6/500
4/4 [==============================] - 0s 7ms/step - loss: 2.3373 - accuracy: 0.4294 - val_loss: 2.5122 - val_accuracy: 0.3415
Epoch 7/500
4/4 [==============================] - 0s 6ms/step - loss: 2.2996 - accuracy: 0.4479 - val_loss: 2.4872 - val_accuracy: 0.3415
Epoch 8/500
4/4 [=========

4/4 [==============================] - 0s 5ms/step - loss: 1.4068 - accuracy: 0.6687 - val_loss: 1.6881 - val_accuracy: 0.5854
Epoch 60/500
4/4 [==============================] - 0s 6ms/step - loss: 1.3985 - accuracy: 0.6626 - val_loss: 1.6732 - val_accuracy: 0.6098
Epoch 61/500
4/4 [==============================] - 0s 6ms/step - loss: 1.3907 - accuracy: 0.6626 - val_loss: 1.6757 - val_accuracy: 0.5854
Epoch 62/500
4/4 [==============================] - 0s 6ms/step - loss: 1.3793 - accuracy: 0.6626 - val_loss: 1.6677 - val_accuracy: 0.5854
Epoch 63/500
4/4 [==============================] - 0s 6ms/step - loss: 1.3703 - accuracy: 0.6748 - val_loss: 1.6657 - val_accuracy: 0.5854
Epoch 64/500
4/4 [==============================] - 0s 6ms/step - loss: 1.3623 - accuracy: 0.6810 - val_loss: 1.6655 - val_accuracy: 0.5854
Epoch 65/500
4/4 [==============================] - 0s 6ms/step - loss: 1.3531 - accuracy: 0.6810 - val_loss: 1.6507 - val_accuracy: 0.6098
Epoch 66/500
4/4 [===============

4/4 [==============================] - 0s 5ms/step - loss: 1.0653 - accuracy: 0.7423 - val_loss: 1.4297 - val_accuracy: 0.6341
Epoch 118/500
4/4 [==============================] - 0s 5ms/step - loss: 1.0617 - accuracy: 0.7423 - val_loss: 1.4369 - val_accuracy: 0.6341
Epoch 119/500
4/4 [==============================] - 0s 6ms/step - loss: 1.0603 - accuracy: 0.7423 - val_loss: 1.4345 - val_accuracy: 0.6341


In [69]:
%load_ext tensorboard
%tensorboard --logdir="logs1"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
mp=model1.predict(testnew1)
print((mp))

[[0. 0. 1. 1. 0. 0.]]


In [71]:
print('symboling value of the test point using model with regularization is',np.argmax(mp)-2)

symboling value of the test point using model with regularization is 0
